In [1]:
######################
# Process MIDI Files #
######################

from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note
import glob
import json
import os

# VARS
DATA_GLOB = "data/pokemon-single/PkmRB/*.mid"
MODEL_DIR = "model/pokemon-single/PkmRB/S120-L256-L256/"
NOTES_FILE = "%s/%s" % (MODEL_DIR, "notes.json")
NOTES_INPUT_PARSED_FILE = "%s/%s" % (MODEL_DIR, "notes-input-parsed.json")

NOTES_BY_FILE = []
NOTES = set()

for input_file in glob.glob(DATA_GLOB):
    print("Ingesting... %s" % input_file)
    
    midi = converter.parse(input_file)
    notes_to_parse = midi.flat.notes
    
    file_notes = []
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            file_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            file_notes.append('.'.join(str(n) for n in element.normalOrder))
    NOTES_BY_FILE.append(file_notes)
    NOTES.update(set(file_notes))

NOTES = sorted(NOTES)

if not os.path.exists(os.path.dirname(NOTES_FILE)):
    os.makedirs(os.path.dirname(NOTES_FILE))
with open(NOTES_FILE, 'w') as note_file:
    json.dump(NOTES, note_file)
with open(NOTES_INPUT_PARSED_FILE, 'w') as notes_input_parsed_file:
    json.dump(NOTES_BY_FILE, notes_input_parsed_file)

Ingesting... data/pokemon-single/PkmRB/PkmRB-Item2.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Indigo.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Center.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Cinnabar.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Item3.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Evolving.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Item.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Celadon.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Title.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-MtMoon.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Oak.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Around.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Rival.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-12.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Cerulean.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Intro.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-10.mid
Ingesting... data/pokemon-single/PkmRB/PkmRB-Vermilion.mid


In [2]:
########################
# Build Note Sequences #
########################

import numpy
from keras.utils import np_utils

NOTES_FILE = None
SEQUENCE_LENGTH = 120
X = []
y = []

# Load Notes from disk if desired
if NOTES_FILE:
    with open(NOTES_FILE, 'r') as notes_file:
        NOTES = json.loads(notes_file.read())

NOTE_TO_INT = dict((note, number) for number, note in enumerate(NOTES))
INT_TO_NOTE = dict((number, note) for number, note in enumerate(NOTES))

for notes in NOTES_BY_FILE:
    for i in range(0, len(notes) - SEQUENCE_LENGTH, 1):
        input_sequence = notes[i:i + SEQUENCE_LENGTH] # N Input Notes
        output_sequence = notes[i + SEQUENCE_LENGTH] # 1 Output Note
        
        X.append([NOTE_TO_INT[c] for c in input_sequence])
        y.append(NOTE_TO_INT[output_sequence])
        
N_SEQUENCES = len(X)
N_VOCAB = len(NOTES)

print("Num Sequences: %d" % N_SEQUENCES)
print("Num Vocab: %d" % N_VOCAB)

X = numpy.reshape(X, (N_SEQUENCES, SEQUENCE_LENGTH, 1)) / N_VOCAB
y = np_utils.to_categorical(y)

/Users/bbamsch/.virtualenvs/cmpe257/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Num Sequences: 3809
Num Vocab: 50


In [3]:
#######################
# Construct the Model #
#######################

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

CHECKPOINT_FILE = None

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(N_VOCAB))
model.add(Activation('softmax'))
if (CHECKPOINT_FILE):
    model.load_weights(CHECKPOINT_FILE)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
###################
# Train the Model #
###################

from keras.callbacks import ModelCheckpoint

MODEL_TEMPLATE = "S120-L256-L256-weights-improvements-{epoch:03d}-{loss:.4f}.hdf5"
filepath = "%s/%s" % (MODEL_DIR, MODEL_TEMPLATE)
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True)
callbacks_list = [checkpoint]

model.fit(X, y, epochs=500, batch_size=128, callbacks=callbacks_list)

Epoch 1/500
3809/3809 [==============================] - 69s 18ms/step - loss: 3.6033
Epoch 2/500
3809/3809 [==============================] - 63s 17ms/step - loss: 3.4929
Epoch 3/500
3809/3809 [==============================] - 66s 17ms/step - loss: 3.4927
Epoch 4/500
3809/3809 [==============================] - 61s 16ms/step - loss: 3.4722
Epoch 5/500
3809/3809 [==============================] - 72s 19ms/step - loss: 3.3977
Epoch 6/500
3809/3809 [==============================] - 66s 17ms/step - loss: 3.6877
Epoch 7/500
3809/3809 [==============================] - 62s 16ms/step - loss: 3.4936
Epoch 8/500
3809/3809 [==============================] - 66s 17ms/step - loss: 3.4833
Epoch 9/500
3809/3809 [==============================] - 63s 17ms/step - loss: 3.4828
Epoch 10/500
3809/3809 [==============================] - 67s 17ms/step - loss: 3.4805
Epoch 11/500
3809/3809 [==============================] - 73s 19ms/step - loss: 3.4822
Epoch 12/500
3809/3809 [============================

3809/3809 [==============================] - 41s 11ms/step - loss: 1.2841
Epoch 96/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.2537
Epoch 97/500
3809/3809 [==============================] - 42s 11ms/step - loss: 1.2527
Epoch 98/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.2133
Epoch 99/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.2204
Epoch 100/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.1909
Epoch 101/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.1532
Epoch 102/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.2663
Epoch 103/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.2718
Epoch 104/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.2182
Epoch 105/500
3809/3809 [==============================] - 41s 11ms/step - loss: 1.1104
Epoch 106/500
3809/3809 [=========================